<a href="https://colab.research.google.com/github/GiliardGodoi/tj-datasets/blob/main/notebooks/01_exemplo_processamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemplo de processamento

## Instalar algumas dependências

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install git+https://github.com/GiliardGodoi/tj-datasets.git --quiet
!git clone https://github.com/GiliardGodoi/tj-datasets.git

Cloning into 'tj-datasets'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 57 (delta 17), reused 44 (delta 8), pack-reused 0
Receiving objects: 100% (57/57), 14.22 KiB | 14.22 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [3]:
# %load_ext autoreload
# !pip install --editable tj-datasets/
!pip install tj-datasets/

Processing ./tj-datasets
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tjdatasets: filename=tjdatasets-0.0.1-py3-none-any.whl size=6354 sha256=28f73e9a78d63e529906fddc89d45fe21a90484fc90c3f857b37847ded4f1ee5
  Stored in directory: /root/.cache/pip/wheels/f5/ba/3e/0a13642d65b485b43309f63c186499fedfecb9e5c04804143e
Successfully built tjdatasets


In [4]:
%%capture
# !python -m spacy download pt_core_news_sm
!python -m spacy download pt_core_news_lg

## Importing libs

In [5]:
import numpy as np
import pandas as pd
import re
import os

from pathlib import Path

In [6]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import RSLPStemmer

nltk.download('punkt')
nltk.download('rslp')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
import spacy
from spacy.lang.pt.stop_words import STOP_WORDS as stop_words_spacy

In [8]:
# nlp = spacy.load("pt_core_news_sm")
nlp = spacy.load("pt_core_news_lg")

In [10]:
# %autoreload

from tjdatasets.preprocessing import (
    remove_punctuation_text,
    remove_word_stress,
    regularize_expressions,
    tokenizer,
    regex_tokenizer,
    lemmatize,
    stemmerize)

from tjdatasets.utils import remove_header_footer, remove_header, remove_footer

## Environment configuration

In [11]:
BASE_FILE = Path("/content/drive/Shareddrives/Projeto_TJSP/Projeto Análise de Precedentes/Datasets/BASE_AGO_2023_STI/acordaos_principais_14k_3_F_limpo.csv")

## Load the data

In [12]:
df = pd.read_csv(BASE_FILE)

In [13]:
df.head(11)

,numero_processo,id_documento,data_hora_documento,natureza_processo,codigos_movimentos_temas,conteudo,formatado
0,00000623620098260063,5564515 - 0,2012-03-16 00:00:00,F,85409;85456;85721;89962,PODER JUDICIÁRIO\nTRIBUNAL DE JUSTIÇA DO ESTAD...,"Registro: 2012.0000103580 ACÓRDÃO Vistos, rela..."
1,00000626520138260590,71696742 - 0,2019-09-02 00:00:00,F,85458;85557;85568;85629,"Es Po PODER JUDICIÁRIO\nmm Ea, TRIBUNAL DE JUS...","mm Ea, TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PA..."
2,00000698620138260063,70131435 - 0,2019-08-02 00:00:00,F,85557;85568;85629;85630,mA E je PODER JUDICIÁRIO\num RE TRIBUNAL DE JU...,um RE TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAU...
3,00000799720148260095,35724068 - 0,2016-11-09 00:00:00,F,80718;85721,PODER JUDICIÁRIO\nTRIBUNAL DE JUSTIÇA DO ESTAD...,"Registro: 2016.0000822015 ACÓRDÃO Vistos, rela..."
4,00000911820138260008,14345586 - 0,2014-06-04 00:00:00,F,85568,PODER JUDICIÁRIO .\nTRIBUNAL DE JUSTIÇA DO EST...,"Registro:2014.0000339672 ACÓRDÃO Vistos, relat..."
5,00000997120138260597,27930183 - 0,2016-02-17 00:00:00,F,85568,== PODER JUDICIÁRIO\n\nTRIBUNAL DE JUSTIÇA DO ...,"Registro: 2016.0000071226 ACÓRDÃO Vistos, rela..."
6,00001024919968260588,72408151 - 0,2019-09-10 00:00:00,F,85728;85763,PODER JUDICIÁRIO\nTRIBUNAL DE JUSTIÇA DO ESTAD...,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO Reg...
7,00001058520128260024,26405769 - 0,2015-12-10 00:00:00,F,80718;85721,"TRIBUNAL DE JUSTIÇA\n\n- —um Dad ,\n== PODER J...","Registro: 2015.0000932221 ACÓRDÃO Vistos, rela..."
8,00001099320138260572,46531774 - 0,2017-11-09 00:00:00,F,80718;85409;85721,-— — PODER JUDICIÁRIO\n= »\n= GET TRIBUNAL DE ...,"Registro: 2017.0000862214 ACÓRDÃO Vistos, rela..."
9,00001104720138260356,17505216 - 0,2014-10-30 00:00:00,F,85557;85568,PODER JUDICI...,"Registro: 2014.0000700338 ACÓRDÃO Vistos, rel..."


In [14]:
df_tmp = df.sample(10)[['numero_processo', 'conteudo']].copy()
# df_tmp = df.loc[:10][['numero_processo', 'conteudo']].copy()

In [15]:
nlp = spacy.load("pt_core_news_lg")

In [17]:
def naive_sentence_segmenter(text: str):
    return re.split(r'\. ', text)

def remove_some_punctuation(text:str, punctuation=r'[—”“\-º°ª]'):
    return re.sub(punctuation, '', text)

# def remove_some_punctuation(text:str):
#     return re.sub(r'[—”“]', '', text)

cabeçalho
- remover cabeçalho e rodapé
- pontuação
- stop words
- tokenization

---
- stemming
- lemmatization
-

In [32]:
def remove_noise_from_header(text : str) -> str:
    doc = list()
    for page in re.split(r'\x0c', text):
        page = re.sub(r'^(.*?)(PODER JUDICIÁRIO)', 'PODER JUDICIÁRIO', page, flags=re.DOTALL)
        page = re.sub(r'PODER JUDICIÁRIO.*?TRIBUNAL DE JUSTIÇA', 'PODER JUDICIÁRIO\nTRIBUNAL DE JUSTIÇA', page, flags=re.DOTALL)
        doc.append(page)

    return '\x0c'.join(doc)

In [29]:
df_tmp['formatado'] = (df_tmp['conteudo']
                       .apply(remove_noise_from_header)
                        .apply(remove_header)
                        .apply(remove_footer)
                        .apply(lambda t: re.sub(r'\x0c', '', t))
                        .apply(lambda t: re.sub(r'\s+', ' ', t))
                        # .apply(lambda t: t.lower())
                        # .apply(remove_word_stress)
                        # .apply(remove_some_punctuation, punctuation=r'[—”“]')
                        .apply(remove_punctuation_text)
                        .apply(regularize_expressions)
                        .apply(lambda t: t.strip())
                        # .apply(naive_sentence_segmenter)
                        # .apply(lemmatize, model=nlp)
                        # .apply(lambda s: ' '.join(s))
                        .apply(tokenizer)
                        # .apply(stemmerize, model=None)
                    )

df_tmp

,numero_processo,conteudo,formatado
13021,22626718020198260000,== PODER JUDICIÁRIO\na TRIBUNAL DE JUSTIÇA DO ...,"[Registro, 2020, 0000170746, ACÓRDÃO, Vistos, ..."
13517,40297434320138260224,ms ST pm PODER JUDICIÁRIO\n==\n\nTRIBUNAL DE J...,"[Registro, 2015, 0000044479, ACÓRDÃO, Vistos, ..."
13474,40025720820138260032,pa PODER JUDICIÁRIO\n== TRIBUNAL DE JUSTIÇA DO...,"[12º, Câmara, de, Direito, Privado, Registro, ..."
2228,10005206220208260059,== PODER JUDICIÁRIO\nmm iDE TRIBUNAL DE JUSTIÇ...,"[Registro, 2021, 0000653495, ACÓRDÃO, Vistos, ..."
8384,20373262820218260000,== PODER JUDICIÁRIO\nmm iDE TRIBUNAL DE JUSTIÇ...,"[Registro, 2021, 0000370384, ACÓRDÃO, Vistos, ..."
10937,21596468520188260000,== PODER JUDICIÁRIO\na TRIBUNAL DE JUSTIÇA DO ...,"[Registro, 2018, 0000895354, ACÓRDÃO, Vistos, ..."
5402,10113848420178260506,mm mem PODER JUDICIÁRIO º\npt TRIBUNAL DE JUST...,"[Registro, 2018, 0000271359, ACÓRDÃO, Vistos, ..."
4184,10041103320188260539,== PODER JUDICIÁRIO\na TRIBUNAL DE JUSTIÇA DO ...,"[Registro, 2019, 0000832138, ACÓRDÃO, Vistos, ..."
10877,21568808820208260000,== PODER JUDICIÁRIO\nmm TRIBUNAL DE JUSTIÇA DO...,"[Registro, 2020, 0000747923, ACÓRDÃO, Vistos, ..."
13137,22681880320188260000,== PODER JUDICIÁRIO\nmm iDE TRIBUNAL DE JUSTIÇ...,"[Registro, 2021, 0000404146, ACÓRDÃO, Vistos, ..."


In [31]:
df_tmp.iloc[3]['formatado']

['Registro',
 '2021',
 '0000653495',
 'ACÓRDÃO',
 'Vistos',
 'relatados',
 'e',
 'discutidos',
 'estes',
 'autos',
 'de',
 'Apelação',
 'Cível',
 'nº',
 '1000520',
 '62',
 '2020',
 '8',
 '26',
 '0059',
 'da',
 'Comarca',
 'de',
 'Bananal',
 'em',
 'que',
 'é',
 'apelante',
 'UNIMED',
 'DE',
 'VOLTA',
 'REDONDA',
 'COOPERATIVA',
 'DE',
 'TRABALHO',
 'MEDICO',
 'é',
 'apelada',
 'MARILEIDE',
 'RODRIGUES',
 'DE',
 'LIMA',
 'ACORDAM',
 'em',
 'sessão',
 'permanente',
 'e',
 'virtual',
 'da',
 '8º',
 'Câmara',
 'de',
 'Direito',
 'Privado',
 'do',
 'Tribunal',
 'de',
 'Justiça',
 'de',
 'são_paulo',
 'proferir',
 'a',
 'seguinte',
 'decisão',
 'Negaram',
 'provimento',
 'ao',
 'recurso',
 'V',
 'U',
 'de',
 'conformidade',
 'com',
 'o',
 'voto',
 'do',
 'relator',
 'que',
 'integra',
 'este',
 'acórdão',
 'O',
 'julgamento',
 'teve',
 'a',
 'participação',
 'dos',
 'Desembargadores',
 'THEODURETO',
 'CAMARGO',
 'Presidente',
 'sem',
 'voto',
 'PEDRO',
 'DE',
 'ALCANTARA',
 'DA',
 'SILVA',
 